# 問3

## コード

### シミュレーションモデル

In [ ]:
import simpy
import numpy as np

class Model:
    _arrivals = []
    _durationQueues, _durationServices = [], []

    def __init__(self, S, debug=False):
        self._env = simpy.Environment()
        self._capacity = simpy.Resource(self._env, capacity=S)
        self._env.process(self._arrival())
        self._debug = debug

    def arrivalTime(self):
        # オーバーライド前提なのでテキトーです。
        return 0.0

    def serviceTime(self):
        # オーバーライド前提なのでテキトーです。
        return 0.0

    def _arrival(self):
        customerId = 0
        while True:
            yield self._env.timeout(self.arrivalTime())
            arrivalTime = self._env.now
            customerId += 1
            self._arrivals.append(arrivalTime)
            if self._debug:
                print("{:.0f} 人目が到着 {:2f}".format(customerId, arrivalTime))
            self._env.process(self.operation(customerId, arrivalTime))

    def _beforeQueue(self, customerId):
        queueIn = self._env.now
        if self._debug:
            print("{:.0f} 人目が並び始めた {:.2f}".format(customerId, queueIn))
        return queueIn

    def _afterQueue(self, customerId):
        queueOut = self._env.now
        if self._debug:
            print("{:.0f} 人目がサービスに入った {:.2f}".format(customerId, queueOut))
        return queueOut

    def _departure(self, customerId):
        departureTime = self._env.now
        if self._debug:
            print("{:.0f} 人目が出発した {:.2f}".format(customerId, departureTime))
        return departureTime

    def tally(self, arrivalTime, queueIn, queueOut, departureTime):
        serviceTime = departureTime - queueOut
        self._durationServices.append(serviceTime)
        queueTime = queueOut - arrivalTime
        self._durationQueues.append(queueTime)

    def operation(self, customerId, arrivalTime):
        with self._capacity.request() as req:
            queueIn = self._beforeQueue(customerId)
            yield req
            queueOut = self._afterQueue(customerId)

            yield self._env.timeout(self.serviceTime())
            departureTime = self._departure(customerId)

            self.tally(arrivalTime, queueIn, queueOut, departureTime)

    def run(self, until):
        self._env.run(until=until)

    def checkResult(self):
        return {
            "arrival": np.mean(np.diff(self._arrivals)),
            "queue": np.mean(self._durationQueues),
            "service": np.mean(self._durationServices),
        }

### 理論値計算

In [ ]:
def theorotical(s, lam, mu):
    rho = lam / (s * mu)
    As = 1
    B = 1
    for n in range(1, s + 1):
        As = As * (s * rho / n)
        B = B + As

    p0 = 1 / (B + As * rho / (1 - rho))
    Lq = rho / ((1 - rho) ** 2) * As * p0
    wq = Lq / lam

    return wq

### M/M/s/∞モデルの定義とプログラムの実行

In [ ]:
class MMsInf(Model):
    before = 0.0

    def __init__(self, s, lam, mu):
        super().__init__(s)
        self._lam = lam
        self._mu = mu

    def arrivalTime(self):
        return np.random.exponential(1 / self._lam)

    def serviceTime(self):
        return np.random.exponential(1 / self._mu)


s = 1
lam = 1
mu = 2

poEx = MMsInf(s, lam, mu)
poEx.run(100000)
results1 = poEx.checkResult()
print("シミュレーションの平均到着間隔:", results1["arrival"])
print("シミュレーションの平均待ち時間:", results1["queue"])
print("待ち時間の理論値",theorotical(s, lam, mu))

## G(一様分布)/M/s/∞モデル

In [ ]:
class UnExSInf(Model):
    def __init__(self, s, lam, mu):
        super().__init__(s)
        self._lam = lam
        self._mu = mu

    def arrivalTime(self):
        ave = 1 / self._lam
        return np.random.uniform(ave - ave * 0.5, ave + ave * 0.5)

    def serviceTime(self):
        return np.random.exponential(1 / self._mu)

s = 1
lam = 5
mu = 10

unEx = UnExSInf(s, lam, mu)
unEx.run(100000)
results2 = unEx.checkResult()
print("平均到着間隔:", results2["arrival"])
print("シミュレーションの平均待ち時間:", results2["queue"])
print("待ち時間の理論値",theorotical(s, lam, mu))

## G(カイ二乗分布)/M/s/∞モデル

In [ ]:
class ChiExSInf(Model):
    def __init__(self, s, lam, mu):
        super().__init__(s)
        self._lam = lam
        self._mu = mu

    def arrivalTime(self):
        return np.random.chisquare(1 / self._lam)

    def serviceTime(self):
        return np.random.exponential(1 / self._mu)

s = 1
lam = 5
mu = 10

chiEx = ChiExSInf(s, lam, mu)
chiEx.run(100000)
results3 = chiEx.checkResult()
print("平均到着間隔:", results3["arrival"])
print("シミュレーションの平均待ち時間:", results3["queue"])
print("待ち時間の理論値", theorotical(s, lam, mu))

## M/G(一様分布)/S/∞モデル

In [ ]:
class PoUniSInf(Model):
    def __init__(self, s, lam, mu):
        super().__init__(s)
        self._lam = lam
        self._mu = mu

    def arrivalTime(self):
        return np.random.poisson(1 / self._lam)

    def serviceTime(self):
        return np.random.uniform(1 / self._mu - 1 / self._mu * 0.5, 1 / self._mu + 1 / self._mu * 0.5)


s = 1
lam = 5
mu = 10

poUniEx = PoUniSInf(s, lam, mu)
poUniEx.run(100000)
results4 = poUniEx.checkResult()
print("平均到着間隔:", results4["arrival"])
print("平均サービス時間:", results4["service"])
print("シミュレーションの平均待ち時間:", results4["queue"])
print("待ち時間の理論値", theorotical(s, lam, mu))

## M/G(カイ二乗分布)/S/∞モデル

In [ ]:
class PoChiSInf(Model):
    def __init__(self, s, lam, mu):
        super().__init__(s)
        self._lam = lam
        self._mu = mu

    def arrivalTime(self):
        return np.random.poisson(1 / self._lam)

    def serviceTime(self):
        return np.random.chisquare(1 / self._mu)

s = 1
lam = 5
mu = 10

poChiEx = PoChiSInf(s, lam, mu)
poChiEx.run(100000)
results5 = poChiEx.checkResult()
print("平均到着間隔:", results5["arrival"])
print("平均サービス時間:", results5["service"])
print("シミュレーションの平均待ち時間:", results5["queue"])
print("待ち時間の理論値", theorotical(s, lam, mu))

## G(一様分布)/G(一様分布)/s/∞モデル

In [ ]:
class UniUniSInf(Model):
    def __init__(self, s, lam, mu):
        super().__init__(s)
        self._lam = lam
        self._mu = mu

    def arrivalTime(self):
        return np.random.uniform(1 / self._lam - 1 / self._lam * 0.5, 1 / self._lam + 1 / self._lam * 0.5)

    def serviceTime(self):
        return np.random.uniform(1 / self._mu - 1 / self._mu * 0.5, 1 / self._mu + 1 / self._mu * 0.5)
    
s = 1
lam = 5
mu = 10

uniUniEx = UniUniSInf(s, lam, mu)
uniUniEx.run(100000)
results6 = uniUniEx.checkResult()
print("平均到着間隔:", results6["arrival"])
print("平均サービス時間:", results6["service"])
print("シミュレーションの平均待ち時間:", results6["queue"])
print("待ち時間の理論値", theorotical(s, lam, mu))

### G(カイ二乗分布)/G(カイ二乗分布)/s/∞モデル

In [ ]:
class ChiChiSInf(Model):
    def __init__(self, s, lam, mu):
        super().__init__(s)
        self._lam = lam
        self._mu = mu

    def arrivalTime(self):
        return np.random.chisquare(1 / self._lam)

    def serviceTime(self):
        return np.random.chisquare(1 / self._mu)
    
s = 1
lam = 5
mu = 10

chiChiEx = ChiChiSInf(s, lam, mu)
chiChiEx.run(100000)
results7 = chiChiEx.checkResult()
print("平均到着間隔:", results7["arrival"])
print("平均サービス時間:", results7["service"])
print("シミュレーションの平均待ち時間:", results7["queue"])
print("待ち時間の理論値", theorotical(s, lam, mu))

### G(一様分布)/G(カイ二乗分布)/s/∞モデル

In [ ]:
class UniChiSInf(Model):
    def __init__(self, s, lam, mu):
        super().__init__(s)
        self._lam = lam
        self._mu = mu

    def arrivalTime(self):
        return np.random.uniform(1 / self._lam - 1 / self._lam * 0.5, 1 / self._lam + 1 / self._lam * 0.5)

    def serviceTime(self):
        return np.random.chisquare(1 / self._mu)
    
s = 1
lam = 5
mu = 10

uniChiEx = UniChiSInf(s, lam, mu)
uniChiEx.run(100000)
results8 = uniChiEx.checkResult()
print("平均到着間隔:", results8["arrival"])
print("平均サービス時間:", results8["service"])
print("シミュレーションの平均待ち時間:", results8["queue"])
print("待ち時間の理論値", theorotical(s, lam, mu))

### G(カイ二乗分布)/G(一様分布)/s/∞モデル

In [ ]:
class ChiUniSInf(Model):
    def __init__(self, s, lam, mu):
        super().__init__(s)
        self._lam = lam
        self._mu = mu

    def arrivalTime(self):
        return np.random.chisquare(1 / self._lam)

    def serviceTime(self):
        return np.random.uniform(1 / self._mu - 1 / self._mu * 0.5, 1 / self._mu + 1 / self._mu * 0.5)
    
s = 1
lam = 5
mu = 10

chiUniEx = ChiUniSInf(s, lam, mu)
chiUniEx.run(100000)
results9 = chiUniEx.checkResult()
print("平均到着間隔:", results9["arrival"])
print("平均サービス時間:", results9["service"])
print("シミュレーションの平均待ち時間:", results9["queue"])
print("待ち時間の理論値", theorotical(s, lam, mu))